In [1]:
! pip install requests pandas gspread

In [3]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
from tapdb import TapDBClient
from gsheets import DateSheet
from config import TAPDB_USER, TAPDB_PASS

# target_date = '2024-06-21'
target_date = (date.today() - timedelta(days=1)).isoformat()

tapdb = TapDBClient(TAPDB_USER, TAPDB_PASS, target_date=target_date)

target_date = tapdb.target_date.isoformat()
sheet = DateSheet('[RP_CN] Revenue Report', date=target_date)

In [2]:
# active devices
sheet.change_sheet('Daily Re.')
data = tapdb.get_data('RP CN (PRO)', 'active', dim='os', device=True)

sheet.update_cells('活躍設備\n(tap db)', int(data['DAU_DEVICE'].sum()))
sheet.update_cells('活躍設備\n(android)', int(data.loc['Android', 'DAU_DEVICE']))
data

,key,DAU_DEVICE,newDAU_DEVICE,DAU_DEVICE2,DAU_DEVICE3,DAU_DEVICE7,DAU_DEVICE14,DAU_DEVICE30
key,,,,,,,,
Android,Android,37593,4457,33136,31743,28187,24059,19429
iOS,iOS,5417,1128,4289,4010,3239,2201,11


In [3]:
# new devices
sheet.change_sheet('Daily Re.')
data = tapdb.get_data('RP CN (PRO)', 'source', dim='os', device=True)
sheet.update_cells('新增設備\nTotal\n(Tap DB)', int(data['newDevices'].sum()))
sheet.update_cells('AND新設備\n(Tap DB)', int(data.loc['Android', 'newDevices']))
data


,key,newPlayers,totalCharge,totalChargeAmount,firstChargePlayers,firstChargeAmount,newDevices,convertedDevices,devicesConvertRate,dayRetainedRate1,dayRetainedRate7,chargeRate,firstChargeRate,ARPU,ARPPU
key,,,,,,,,,,,,,,,
Android,Android,4945,56,317.5,31,78.4,4457,3796,85.17,32.51,None,1.26,0.70,0.02,2.53
iOS,iOS,1030,34,622.8,17,97.5,1128,977,86.61,30.15,None,3.01,1.51,0.09,5.74


In [4]:
# convert rate by version
data = tapdb.get_data('RP CN (PRO)', 'source', dim='ver')
row = data.loc[max(data.index)]

sheet.change_sheet('轉化率')
sheet.update_cells('版本', row['key'])
sheet.update_cells('新增設備', row['newDevices'])
sheet.update_cells('轉化設備', row['convertedDevices'])

data

,key,newPlayers,totalCharge,totalChargeAmount,firstChargePlayers,firstChargeAmount,newDevices,convertedDevices,devicesConvertRate,dayRetainedRate1,dayRetainedRate7,chargeRate,firstChargeRate,ARPU,ARPPU
key,,,,,,,,,,,,,,,
8.0.0.353,8.0.0.353,5819,87,933.7,46,172.6,5010,4404,87.90,31.93,None,1.74,0.92,0.03,3.75
8.0.0.352,8.0.0.352,156,3,6.6,2,3.3,225,167,74.22,30.43,None,1.33,0.89,0.01,1.65
1.0.0.146,1.0.0.146,0,0,0.0,0,0.0,170,118,69.41,44.88,None,0.00,0.00,0.00,0.00
7.0.0.325,7.0.0.325,0,0,0.0,0,0.0,138,66,47.83,25.77,None,0.00,0.00,0.00,0.00
6.0.0.274,6.0.0.274,0,0,0.0,0,0.0,13,5,38.46,27.27,None,0.00,0.00,0.00,0.00
7.0.0.308,7.0.0.308,0,0,0.0,0,0.0,11,6,54.55,30.43,None,0.00,0.00,0.00,0.00
4.0.0.211,4.0.0.211,0,0,0.0,0,0.0,4,3,75.00,50.00,None,0.00,0.00,0.00,0.00
6.0.0.273,6.0.0.273,0,0,0.0,0,0.0,4,0,0.00,NaN,None,0.00,0.00,0.00,0.00
5.0.0.256,5.0.0.256,0,0,0.0,0,0.0,3,1,33.33,20.00,None,0.00,0.00,0.00,0.00


In [111]:
# retain
data = tapdb.get_data('RP CN (PRO)', 'retain', begin_date='2024-04-01', pf='and', type='both')
data.sort_index(inplace=True)
data['key'] = data['key'].astype(str)
data_drr = data.loc[:, data.columns.str.contains('DRR')]
data = pd.concat([data.loc[:, ['newDevices']], data_drr / 100], axis=1)

data.to_csv('~/Desktop/retain.csv')
data

,newDevices,DRR1,DRR2,DRR3,DRR4,DRR5,DRR6,DRR7,DRR8,DRR9,...,DRR26,DRR27,DRR28,DRR29,DRR30,DRR60,DRR90,DRR120,DRR150,DRR180
key,,,,,,,,,,,,,,,,,,,,,
2024-04-01,3392,34.20,25.03,21.14,18.16,16.89,15.18,14.56,13.44,12.03,...,6.04,5.96,5.66,5.22,5.45,2.39,None,None,None,None
2024-04-02,3796,33.11,23.23,19.55,17.52,16.10,14.54,13.94,12.62,11.17,...,5.37,5.22,5.48,4.93,4.69,2.24,None,None,None,None
2024-04-03,6546,32.62,19.75,15.73,12.57,11.21,10.59,9.30,8.66,8.80,...,4.09,5.30,5.27,4.66,3.93,1.82,None,None,None,None
2024-04-04,10555,29.28,17.51,11.22,10.22,9.68,8.53,7.98,7.94,9.50,...,5.34,6.26,4.74,4.18,3.94,1.24,None,None,None,None
2024-04-05,8652,28.51,14.72,12.91,11.53,10.66,9.93,10.53,11.92,9.87,...,6.24,5.61,5.20,4.74,4.52,1.70,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-18,4007,36.21,24.48,21.24,18.22,16.50,14.97,9.58,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
2024-06-19,3911,36.38,26.23,22.48,18.82,17.39,11.63,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
2024-06-20,3888,35.83,23.95,20.32,17.46,10.93,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
